Load Data

In [123]:
import pandas as pd

kode_saham = ["PBCRF","BKRKF","PHJMF","PTAIF","TLKMF"]
n = len(kode_saham)
github = "https://raw.githubusercontent.com/relfarizi/Portfolio-Saham/master/"
#data[0] = "https://raw.githubusercontent.com/relfarizi/Portfolio-Saham/master/"+data[0]+".csv"
#data[1] = "https://raw.githubusercontent.com/relfarizi/Portfolio-Saham/master/BKRKF.csv"
#data[2] = "https://raw.githubusercontent.com/relfarizi/Portfolio-Saham/master/PHJMF.csv"
#data[3] = "https://raw.githubusercontent.com/relfarizi/Portfolio-Saham/master/PTAIF.csv"
#data[4] = "https://raw.githubusercontent.com/relfarizi/Portfolio-Saham/master/TLKMF.csv"

df = []
for i in range(n) :
  datas = pd.read_csv(github+kode_saham[i]+".csv")
  data_tmp = datas["Close"]
  df.append(data_tmp)
  #print(data_tmp.shape)

dataset = pd.concat(df, axis=1, sort=False)
dataset.columns = data
dataset

,PBCRF,BKRKF,PHJMF,PTAIF,TLKMF
0,1.13,0.176,0.31,0.593,0.27
1,1.13,0.186,0.31,0.593,0.29
2,1.13,0.172,0.31,0.593,0.28
3,1.05,0.172,0.29,0.593,0.26
4,1.12,0.178,0.29,0.593,0.28
...,...,...,...,...,...
100,1.70,0.250,0.24,0.480,0.25
101,1.73,0.240,0.25,0.480,0.27
102,1.70,0.250,0.25,0.480,0.26
103,1.73,0.260,0.25,0.480,0.26


In [124]:
dataset = dataset.dropna()
#print(type(dataset.corr()))
list_corr = []
for x in dataset.corr():
  list_corr.append(dataset.corr()[x].tolist())
  print(dataset.corr()[x].tolist())
rho = np.array(list_corr)
print(rho)

[1.0, 0.7615901719657908, 0.178278889112772, -0.5089427877343716, -0.3853462162916537]
[0.7615901719657908, 1.0, 0.5169923000605561, -0.05432480430708324, -0.10288364911157866]
[0.178278889112772, 0.5169923000605561, 1.0, 0.3196164202786966, -0.09471490216885914]
[-0.5089427877343716, -0.05432480430708324, 0.3196164202786966, 1.0, 0.5777326173034627]
[-0.3853462162916537, -0.10288364911157866, -0.09471490216885914, 0.5777326173034627, 1.0]
[[ 1.          0.76159017  0.17827889 -0.50894279 -0.38534622]
 [ 0.76159017  1.          0.5169923  -0.0543248  -0.10288365]
 [ 0.17827889  0.5169923   1.          0.31961642 -0.0947149 ]
 [-0.50894279 -0.0543248   0.31961642  1.          0.57773262]
 [-0.38534622 -0.10288365 -0.0947149   0.57773262  1.        ]]


In [125]:
import statistics as st
var = []
for x in dataset :
  tmp = dataset[x].tolist()
  var.append(st.variance(tmp))
print(var)

[0.034232571228528755, 0.00079821359223301, 0.0010388256161314412, 0.0034914524635922335, 0.0019262135922330098]


# Multi sekuritas


In [126]:
import numpy as np

std_dev = []
for i in var :
  std_dev.append(np.sqrt(i))
n = len(std_dev)

u = np.ones(n)
#print(u)
#rho = np.zeros((n,n))
#rho[0][1] = 0.1

#rho[0][2] = 0.2
#rho[0][3] = 0.2
#rho[0][4] = 0.2
#rho[1][2] = 0.2
#rho[1][3] = 0.2
#rho[1][4] = 0.2
#rho[2][3] = 0.2
#rho[2][4] = 0.2
#rho[3][4] = 0.2

for i in range(n) :
  for j in range(n) :
    if j == i :
      rho[i][j] = 1
    elif j < i :
      rho[i][j] = rho[j][i]
print(rho)

[[ 1.          0.76159017  0.17827889 -0.50894279 -0.38534622]
 [ 0.76159017  1.          0.5169923  -0.0543248  -0.10288365]
 [ 0.17827889  0.5169923   1.          0.31961642 -0.0947149 ]
 [-0.50894279 -0.0543248   0.31961642  1.          0.57773262]
 [-0.38534622 -0.10288365 -0.0947149   0.57773262  1.        ]]


In [127]:
print(rho[0][1])

0.7615901719657908


In [0]:
def is_short_selling(rho,std_dev):
  n = len(rho)
  print(n)
  check = False
  #rho = rho.sort()
  std_Dev =std_dev.sort()

  for i in range(n):
    for j in range(n):
      if i < j :
        x = std_dev[i]/std_dev[j]
        #print(rho[i][j])
        if rho[i][j] > x :
          check =True
          break
  return check

In [129]:
print(rho)

[[ 1.          0.76159017  0.17827889 -0.50894279 -0.38534622]
 [ 0.76159017  1.          0.5169923  -0.0543248  -0.10288365]
 [ 0.17827889  0.5169923   1.          0.31961642 -0.0947149 ]
 [-0.50894279 -0.0543248   0.31961642  1.          0.57773262]
 [-0.38534622 -0.10288365 -0.0947149   0.57773262  1.        ]]


In [130]:
is_short_selling(rho,std_dev)

5


True

In [131]:
C = (np.zeros((n,n)))

for i in range(n):
  for j in range(n) :
    C[i][j] = std_dev[i]*rho[i][j]*std_dev[j]
print(C)

[[ 0.00079821  0.00069351  0.00022106 -0.00084963 -0.00201433]
 [ 0.00069351  0.00103883  0.00073132 -0.00010346 -0.00061353]
 [ 0.00022106  0.00073132  0.00192621  0.00082887 -0.00076911]
 [-0.00084963 -0.00010346  0.00082887  0.00349145  0.00631611]
 [-0.00201433 -0.00061353 -0.00076911  0.00631611  0.03423257]]


In [132]:
from numpy.linalg import inv
from numpy.linalg import det
u = np.ones(n)
uT = np.transpose(u)

CInv = inv(C)

w = np.dot(u,CInv)/(u.dot(CInv).dot(uT))
print(w)

[ 1.0847596  -0.47011831  0.08240948  0.29100983  0.0119394 ]


In [133]:
#w = np.array([0.3,0.4])
wT = np.transpose(w)
x = np.dot(u,wT)
print("jumlah w = ",x)

jumlah w =  1.0


In [134]:
risk = w.dot(C).dot(wT)
print(risk)

0.0002867544304543157


Expected return saham

In [0]:
import math
def find_ud(saham) :
  K = []
  
  
  for i in range(len(saham)-1) :
    x = (saham[i+1]-saham[i])/saham[i]
    K.append(x)
  up = []
  down = []

  for i in K :
    if i > 0 :
      x = 1
      y = 0
    else :
      x = 0
      y = 1
    up.append(x)
    down.append(y)
  
  x_up = 1
  x_down = 1

  sum_u = 0
  sum_d = 0

  for i in range(len(K)) :
    x_up += K[i]*up[i]
    sum_u += up[i]
    x_down += K[i]*down[i]
    sum_d += down[i]
  
  u = x_up/sum_u
  d = x_up/sum_d

  p = sum_u/(sum_u+sum_d)
  q = 1-p

  result = {"u" : u, "d" : d,"p":p,"q":q}
  
  return result

In [136]:
list_saham = dataset["BKRKF"]
find_ud(list_saham)

{'d': 0.057387452271193726,
 'p': 0.4174757281553398,
 'q': 0.5825242718446602,
 'u': 0.08007551479701451}

In [0]:
def return_saham(df_saham) :
  result = {}
  for saham in df_saham :
    x = find_ud(df_saham[saham])
    exp = (x["u"]*x["p"])+(x["d"]*x["q"])
    result[saham] = exp
  return result

In [0]:
return_ = return_saham(dataset)
m = []
for saham in return_ :
  m.append(return_[saham])

In [140]:

import pandas as pd

m = np.array(m)
print(m)
EK = m.dot(wT)
print("ekspektasi return = ", EK)
risk = w.dot(C).dot(wT)
print("risk = ",risk)

portfolio=[round(x,4) for x in w]
portfolio.append(EK)
portfolio.append(risk)
print(portfolio)

df_port = pd.DataFrame([portfolio])
column = kode_saham
print(kode_saham)
column.append("Expected Return")
column.append("Risiko")
df_port.columns = column
df_port

[0.04988471 0.06685917 0.0321097  0.03255372 0.08804206]
ekspektasi return =  0.03585195944661085
risk =  0.0002867544304543157
[1.0848, -0.4701, 0.0824, 0.291, 0.0119, 0.03585195944661085, 0.0002867544304543157]
['PBCRF', 'BKRKF', 'PHJMF', 'PTAIF', 'TLKMF']


,PBCRF,BKRKF,PHJMF,PTAIF,TLKMF,Expected Return,Risiko
0,1.0848,-0.4701,0.0824,0.291,0.0119,0.035852,0.000287


In [90]:
#mT = np.transpose(m)
e_r = 0.03
matA = np.dot(u,(CInv))
matB = np.dot(m,(CInv))
mat1 = u.dot(CInv).dot(mT)
mat2 = m.dot(CInv).dot(mT)
mat3 = u.dot(CInv).dot(uT)
mat4 = m.dot(CInv).dot(uT)

A1 = np.array([[1,mat1],[e_r,mat2]]) 
A2 = np.array([[mat3,1],[mat4,e_r]])
A3 = np.array([[mat3,mat1],[mat4,mat2]])

det1 = det([[1,mat1],[e_r,mat2]])
det2 = det([[mat3,1],[mat4,e_r]])
det3 = det([[mat3,mat1],[mat4,mat2]])
w_new = ((det1*matA)+(det2*matB))/det3
print(w_new)

NameError: ignored

In [0]:
import math



sigma1_2 = 0.04
sigma1 = math.sqrt(sigma1_2) 

#sigma1 = 
#sigma1_2 = pow(sigma1,2)

sigma2_2 = 0.09 
sigma2 = math.sqrt(sigma2_2)

#sigma2 =
#sigma2_2 = pow(sigma2,2)
rho = 0.1

x = (2*rho*sigma1*sigma2)
w2 = (sigma1_2 - x)/(sigma1_2+sigma2_2-x)
w1 = 1-w2

w2 = round(w2,4) 
w1 = round(w1,4)

sigmav_2 = (pow(w1,2)*sigma1_2) + (pow(w2,2)*sigma2_2) + (2*w1*w1*rho*sigma1*sigma2)
sigmav_2 = round(sigmav_2,4)


#short selling
# asumsi diberikan sigma1 > sigma2

if rho >= (sigma1/sigma2) :
  print("short selling")
else :
  print("tidak shoort selling")
print(w1,w2,sigmav_2)

tidak shoort selling
0.7627 0.2373 0.0353


In [0]:
# risiko portfolio
risk = w.dot(C).dot(wT)
print("risk = ",risk)

#Expetasi return
ex_return = np.dot(m,wT)
print("expektasi return = ", ex_return)

risk =  0.030203389830508482
expektasi return =  0.025762711864406783


In [0]:
std_dev = [1.5,0.5,1.2]
m = np.array([0.08,0.1,0.06])
n = len(std_deviasi)
rho12 = 0.3
rho23 = 0
rho13 = -0.2

NameError: ignored

In [0]:
c11 = std_deviasi[0]**2
c22 = std_deviasi[1]**2
c33 = std_deviasi[2]**2

c12 = rho12*std_dev[0]*std_dev[1]
c21 = c12

c23 = rho23*std_dev[1]*std_dev[2]
c32 = c23

c13 = rho13*std_dev[0]*std_dev[2]
c31 = c13

C_matrix = np.array([[c11,c12,c13],[c21,c22,c23],[c31,c32,c33]])
print(C_matrix)

In [0]:
from numpy.linalg import inv
from numpy.linalg import det
u = np.ones(n)
uT = np.transpose(u)

CInv = inv(C_matrix)

w = np.dot(u,CInv)/(u.dot(CInv).dot(uT))
#print(w)

for i in range(len(w)) :
  print("w",i+1,"=",w[i])

In [0]:
wT = np.transpose(w)
eksp_return = m.dot(wT)
min_risk = w.dot(C_matrix).dot(wT)
print("ekspektasi return = ",eksp_return)
print("risiko minimum = ", min_risk)

In [0]:
print(A1)
print(A2)
print(A3)

In [0]:
print(w_new)

In [0]:
for i in range(len(w_new)) :
  print("w",i+1,"=",w_new[i])